In [ ]:
# 生成tfrecord
def image2tfrecord(data_list,label_list):
    len2 = len(data_list)
    print("len=",len2)
    writer = tf.python_io.TFRecordWriter(local_path+'%s.tfrecord' %'train')
    for i in range(len2):
        #读取图片并解码
        # image = Image.open(image_list[i])
        # image = image.resize((28,28))
        data = data_list[i]
        #print(data[0])
        #len3 = len(data)
       # for j in range(len3):
        #转化为原始字节
        data_bytes = data[0] # .tobytes()
        #创建字典
        features = {}
        
        tf_example = tf.train.Example(
                features=tf.train.Features(
                    feature={ 'user_id': tf.train.Feature(float_list=tf.train.FloatList(value=[data_bytes]))
                     })) 
    

        #序列化样本
        tf_serialized = tf_example.SerializeToString()
        #将序列化的样本写入rfrecord
        writer.write(tf_serialized)
    writer.close()
    

In [6]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import tensorflow as tf
import numpy
writer = tf.python_io.TFRecordWriter('test1111.tfrecord')
for i in range(0, 2):
    a = 0.618 + i
    b = [2016 + i, 2017+i]
    #c = numpy.array([[0, 1, 2],[3, 4, 5]]) + i
    #c = c.astype(numpy.uint8)
    #c = "你好哦"+str(i)
    #c_raw = c.tostring()#这里是把ｃ换了一种格式存储
    #c_raw = c
    print('i:',i)
    print('  a:',a)
    print('  b:',b)
    #print('  c:',c)
    example = tf.train.Example(
        features = tf.train.Features(
            feature = {'a':tf.train.Feature(float_list = tf.train.FloatList(value=[a])),
                       'b':tf.train.Feature(int64_list = tf.train.Int64List(value = b))
                       #,'c':tf.train.Feature(bytes_list = tf.train.BytesList(value = [c_raw]))
                      }))
    serialized = example.SerializeToString()
    writer.write(serialized)
    print( '   writer',i,'DOWN!')
writer.close()

i: 0
  a: 0.618
  b: [2016, 2017]
   writer 0 DOWN!
i: 1
  a: 1.6179999999999999
  b: [2017, 2018]
   writer 1 DOWN!


In [11]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# output file name string to a queue
filename_queue = tf.train.string_input_producer(['test1111.tfrecord'], num_epochs=None)
# create a reader from file queue
reader = tf.TFRecordReader()
_, serialized_example = reader.read(filename_queue)
# get feature from serialized example


features = tf.parse_single_example(serialized_example,
        features={
            'a': tf.FixedLenFeature([], tf.float32),
            'b': tf.FixedLenFeature([2], tf.int64)
            
        }
    )
a_out = features['a']
b_out = features['b']
#c_out = features['c']
#c_raw_out = features['c']
#c_raw_out = tf.sparse_to_dense(features['c'])
#c_out = tf.decode_raw(c_raw_out, tf.uint8)
print('----', a_out)
print ('----',b_out)
#print c_out
#c_out = tf.reshape(c_out, [2, 3])
a_batch, b_batch = tf.train.shuffle_batch([a_out, b_out], batch_size=3,
                                capacity=200, min_after_dequeue=100, num_threads=2)
sess = tf.Session()
init = tf.initialize_all_variables()
sess.run(init)

tf.train.start_queue_runners(sess=sess)
a_val, b_val = sess.run([a_batch, b_batch])
# print(a_val, b_val, c_val)
print('first batch:') 
print('  a_val:',a_val)
print('  b_val:',b_val) 






---- Tensor("ParseSingleExample_1/ParseSingleExample:0", shape=(), dtype=float32)
---- Tensor("ParseSingleExample_1/ParseSingleExample:1", shape=(2,), dtype=int64)
first batch:
  a_val: [1.618 0.618 1.618]
  b_val: [[2017 2018]
 [2016 2017]
 [2017 2018]]


In [10]:
!python -V

Python 2.7.15 :: Anaconda, Inc.


In [5]:
sys.getdefaultencoding()

NameError: name 'sys' is not defined

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# output file name string to a queue
filename_queue = tf.train.string_input_producer(['test1111.tfrecord'], num_epochs=None)
# create a reader from file queue
reader = tf.TFRecordReader()
_, serialized_example = reader.read(filename_queue)
# get feature from serialized example


features = tf.parse_single_example(serialized_example,
        features={
            'a': tf.FixedLenFeature([], tf.float32),
            'b': tf.FixedLenFeature([2], tf.int64),
            'c': tf.FixedLenFeature([],tf.string)
        }
    )
a_out = features['a']
b_out = features['b']
c_out = features['c']
#c_raw_out = features['c']
#c_raw_out = tf.sparse_to_dense(features['c'])
#c_out = tf.decode_raw(c_raw_out, tf.uint8)
print a_out
print b_out
print c_out
#c_out = tf.reshape(c_out, [2, 3])
a_batch, b_batch, c_batch = tf.train.shuffle_batch([a_out, b_out, c_out], batch_size=3,
                                capacity=200, min_after_dequeue=100, num_threads=2)
sess = tf.Session()
init = tf.initialize_all_variables()
sess.run(init)

tf.train.start_queue_runners(sess=sess)
a_val, b_val, c_val = sess.run([a_batch, b_batch, c_batch])
# print(a_val, b_val, c_val)
print 'first batch:'
print '  a_val:',a_val
print '  b_val:',b_val
print '  c_val:',c_val[0].decode('utf-8')
a_val, b_val, c_val = sess.run([a_batch, b_batch, c_batch])
print 'second batch:'
print '  a_val:',a_val
print '  b_val:',b_val
print '  c_val:',str(c_val).decode('utf-8')




# output file name string to a queue
filename_queue = tf.train.string_input_producer(['/data/nfs/guodapeng/ctr/wd/data/transformed/test/14.tfrecord'], num_epochs=None)
# create a reader from file queue
reader = tf.TFRecordReader()
_, serialized_example = reader.read(filename_queue)
# get feature from serialized example


features = tf.parse_single_example(
        serialized_example,
        features={
            "label": tf.FixedLenFeature([], tf.float32),
            "feature": tf.FixedLenFeature([38], tf.string),
            "times": tf.FixedLenFeature([], tf.float32)
        })

label = features["label"]
feature = features["feature"]
times = features["times"]


sess = tf.Session()
init = tf.initialize_all_variables()
sess.run(init)

tf.train.start_queue_runners(sess=sess)
a_val, b_val, c_val = sess.run([label, features, times])

print 'first batch:'
print '  a_val:',a_val
print '  b_val:',b_val
print '  c_val:',c_val[0].decode('utf-8')